# APIM ❤️ AI Foundry

## Message Storing lab
![flow](../../images/message-storing.gif)

Playground to test storing message details into Cosmos DB. This lab leverages the built-in LLM logging feature to capture prompts, completions, and token counts into Azure Monitor. Data is continuously exported to Event Hub, then ingested into Cosmos DB via Azure Stream Analytics.

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`... 


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the models and versions according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [1]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "uksouth"

aiservices_config = [{"name": "foundry1", "location": "uksouth"}]

models_config = [{"name": "gpt-4.1-mini", "publisher": "OpenAI", "version": "2025-04-14", "sku": "GlobalStandard", "capacity": 20}]

apim_sku = 'Basicv2'

apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}]

inference_api_path = "inference"  # path to the inference API in the APIM service
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

cosmosdb_database_name = "llmdb"
cosmosdb_container_name = "messages"

utils.print_ok('Notebook initialized')



✅ Notebook initialized ⌚ 10:29:14.747572 


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [2]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 10:29:17.455132 :2s]
👉🏽 Current user: lproux@microsoft.com
👉🏽 Tenant ID: 2b9d9f47-1fb6-400a-a438-39fe7d768649
👉🏽 Subscription ID: d334f2cd-3efd-494e-9fd3-2470b1a13e4c


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [3]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "cosmosDBDatabaseName": { "value": cosmosdb_database_name },
        "cosmosDBContainerName": { "value": cosmosdb_container_name },
        "foundryProjectName": { "value": foundry_project_name }
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-message-storing 
👉🏽 Resource group lab-message-storing does not yet exist. Creating the resource group now...
⚙️ Running: az group create --name lab-message-storing --location uksouth --tags source=ai-gateway 
✅ Resource group 'lab-message-storing' created ⌚ 10:29:28.797115 :6s]
⚙️ Running: az deployment group create --name message-storing --resource-group lab-message-storing --template-file main.bicep --parameters params.json 
✅ Deployment 'message-storing' succeeded ⌚ 10:33:51.063348 :22s]


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [4]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key
    app_insights_id = utils.get_deployment_output(output, 'applicationInsightsAppId', 'Application Insights Id')
    cosmosdb_connection_string = utils.get_deployment_output(output, 'cosmosDBConnectionString', 'Cosmos DB Connection String')



⚙️ Running: az deployment group show --name message-storing -g lab-message-storing 
✅ Retrieved deployment: message-storing ⌚ 10:33:55.133655 :4s]
👉🏽 Log Analytics Id: a61b75e2-0f0f-4c9a-813c-c02b633567e2
👉🏽 APIM Service Id: /subscriptions/d334f2cd-3efd-494e-9fd3-2470b1a13e4c/resourceGroups/lab-message-storing/providers/Microsoft.ApiManagement/service/apim-j2o47yhrgbhas
👉🏽 APIM API Gateway URL: https://apim-j2o47yhrgbhas.azure-api.net
👉🏽 Subscription Name: subscription1
👉🏽 Subscription Key: ****6959
👉🏽 Application Insights Id: 986cae35-ccb7-4cae-9cc7-917a56b4add2
👉🏽 Cosmos DB Connection String: AccountEndpoint=https://cosmosdb-j2o47yhrgbhas.documents.azure.com:443/;AccountKey=QRB99PObuHH0wmkMRJAPAZrFtVJg1Sf9Tj7Qwd7dhFTfA9wYYKRxUUC5ze1TdjQGctDERFfPaWkZACDbkHpMcA==;


<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK


In [5]:
import time
from openai import AzureOpenAI
for i in range(10):
    print("▶️ Run: ", i+1)
    
    client = AzureOpenAI(
        azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
        api_key=api_key,
        api_version=inference_api_version
    )
    response = client.chat.completions.create(model=models_config[0]['name'], messages=[
                    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
                    {"role": "user", "content": "Can you tell me the time, please?"}
    ])
    print("💬 ",response.choices[0].message.content)


▶️ Run:  1
💬  Oh sure, let me just magically sync with your local vibes and tell you the time. Or, you know, maybe just look at a clock or your phone like a normal person. But if you really want to know, try turning your head towards the nearest device with a display. Revolutionary stuff!
▶️ Run:  2
💬  Oh sure, let me just tap into the timeless void of the internet and magically know the current time for you. Or, you know, you could just look at a clock. Just a wild idea!
▶️ Run:  3
💬  Oh sure, let me just magically tap into the space-time continuum for you. Or, you know, you could look at the clock on your device. But hey, I’m here to help—just don’t expect me to hand you a watch through the screen!
▶️ Run:  4
💬  Oh sure, let me just tap into the space-time continuum real quick. Or, you know, maybe just look at the clock on your device? But hey, who am I to judge your choice!
▶️ Run:  5
💬  Oh sure, let me just look into my crystal ball and magically tell you the time. Or, you know, ma

<a id='cosmosdb'></a>
### 🔍 Analyze CosmosDB items

Run `pip install azure-cosmos`before executing the following script


In [7]:
import pandas as pd
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey

client = CosmosClient.from_connection_string(conn_str=cosmosdb_connection_string)
database = client.get_database_client(cosmosdb_database_name)
container = database.get_container_client(cosmosdb_container_name)

item_list = list(container.read_all_items(max_item_count=10))

print('Found {0} items'.format(item_list.__len__()))

df = pd.DataFrame(container.query_items('select c.conversationStart, c.id, c.model, c.subscriptionId, c.backendId, c.promptTokens, c.completionTokens, c.totalTokens, c.request, c.response from c order by c.conversationStart desc',enable_cross_partition_query=True))
df

ImportError: cannot import name 'CosmosClient' from 'azure.cosmos' (c:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\.venv\Lib\site-packages\azure\cosmos\__init__.py)

<a id='fabric'></a>
### 📊 Extract insights and visualize data in motion

![real time intelligence](https://learn.microsoft.com/fabric/real-time-intelligence/media/overview/overview-schematic.png)

- Now that we have the data in Event Hub, we can create a Microsoft Fabric [Eventstream](https://learn.microsoft.com/fabric/real-time-intelligence/event-streams/add-source-azure-event-hubs?pivots=enhanced-capabilities) connected to Event Hub to ingest & process, analyze & transform and act on AI usage with [Real-Time Intelligence](https://learn.microsoft.com/fabric/real-time-intelligence/overview).
- You can also use [Cosmos DB mirroring](https://learn.microsoft.com/fabric/database/mirrored-database/azure-cosmos-db) to continuously replicate Cosmos DB data directly into Fabric OneLake in near real-time.

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.